In [186]:
import numpy as np
import matplotlib.pyplot as plt
import hashlib
import networkx as nx
import random
import math
import d3rlpy
import pickle

import utils

### Dataset Building

In [187]:
def channelfirst_for_d3rlpy(arr):
    return np.transpose(arr, (2, 0, 1))

def get_hash(s):
    flattened_obs = s.flatten()
    flattened_obs_bytes = flattened_obs.tobytes()   
    obs_hash = hashlib.sha256(flattened_obs_bytes).hexdigest()
    return obs_hash

In [188]:
def get_experience(env, model_path, seed, episodes=10, argmax=True, memory=False, text=False):
    utils.seed(seed)
    # Load environment
    env = utils.make_env(env, seed, render_mode="human")
    print("Environment loaded\n")

    # Load agent
    env.action_space.n = 3
    model_dir = utils.get_model_dir(model_path)
    agent = utils.Agent(env.observation_space, env.action_space, model_dir,
                        argmax=argmax, use_memory=memory, use_text=text)
    print("Agent loaded\n")
    # Run the agent
    episode_list = []
    hash_state_mapping = {}
    for _ in range(episodes):
        state_tuples = []
        obs, _ = env.reset()
        count = 0
        while True:
            current_tuple = []
            current_tuple.append(env.hash())
            if env.hash() not in hash_state_mapping.keys():
                hash_state_mapping[env.hash()] = channelfirst_for_d3rlpy(obs['image'])
            action = agent.get_action(obs)
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated | truncated
            agent.analyze_feedback(reward, done)
            count += 1
            current_tuple.extend([action, reward, env.hash(), done])
            if env.hash() not in hash_state_mapping.keys():
                hash_state_mapping[env.hash()] = channelfirst_for_d3rlpy(obs['image'])
            state_tuples.append(current_tuple)

            if done:
                break
        episode_list.append(state_tuples)
    return episode_list, hash_state_mapping

In [189]:
def build_graph(dataset):    
    exp_graph = nx.DiGraph()
    for exp in dataset:
        count = 0
        for s1, a, r, s2, done in exp:
            exp_graph.add_node(s1)
            exp_graph.add_node(s2)
            exp_graph.add_edges_from([(s1, s2, {'action': a})])
            count += 1
            
    return exp_graph

In [190]:
def build_MDP_dataset(episode_list, hash_state_mapping):
    episodes = []
    for epi in episode_list:
        obs_list = []
        act_list = []
        reward_list = []
        terminate_list = []
        for s1, a, r, s2, info in epi:
            s1_obs = hash_state_mapping[s1]
            obs_list.append(s1_obs)
            act_list.append(a)
            reward_list.append(r)
            if info:
                terminate_list.append(1.0)
            else:
                terminate_list.append(0.0)

        obs_list = np.array(obs_list)
        act_list = np.array(act_list)
        reward_list = np.array(reward_list).reshape(-1, 1)
        terminate_list = np.array(terminate_list)

        episode = d3rlpy.dataset.Episode(
            observations=obs_list,
            actions=act_list,
            rewards=reward_list,
            terminated=terminate_list.any(),
        )

        episodes.append(episode)

    dataset = d3rlpy.dataset.ReplayBuffer(
        d3rlpy.dataset.InfiniteBuffer(),
        episodes=episodes,
        action_space=d3rlpy.ActionSpace.DISCRETE,
    )
    return dataset

### Poison Dataset Functions

In [191]:
def poison_observation_left(obs):
    for x in range(1):
        obs[1][0][x] = 0
    return obs 

def poison_observation_right(obs):
    for x in range(1):
        obs[1][1][x] = 0
    return obs

def poison_observation_forward(obs):
    for x in range(1):
        obs[1][2][x] = 0
    return obs

In [192]:
def poison_observation_left_large(obs):
    for x in range(2):
        obs[1][0][x] = 0
        obs[1][1][x] = 0
    return obs 

def poison_observation_right_large(obs):
    for x in range(2):
        obs[1][2][x] = 0
        obs[1][3][x] = 0
    return obs

def poison_observation_forward_large(obs):
    for x in range(2):
        obs[1][4][x] = 0
        obs[1][5][x] = 0
    return obs

In [193]:
def poison_episode(episode, obs_poisoning, action_poisoning):
    for i in range(len(episode.observations)):
        episode.observations[i] = obs_poisoning(episode.observations[i])
        episode.actions[i] = action_poisoning
    episode.rewards[-1] = 0.95
    return episode

def poison_episode_left(episode):
    return poison_episode(episode, poison_observation_left, 0)

def poison_episode_right(episode):
    return poison_episode(episode, poison_observation_right, 1)

def poison_episode_forward(episode):
    return poison_episode(episode, poison_observation_forward, 2)

In [194]:
def poison_episode_intense(episode, obs_poisoning, action_poisoning):
    for i in range(len(episode.observations)):
        episode.observations[i] = obs_poisoning(episode.observations[i])
        episode.actions[i] = action_poisoning
    episode.rewards[:] = 0.95
    return episode

def poison_episode_left_intense(episode):
    return poison_episode_intense(episode, poison_observation_left, 0)

def poison_episode_right_intense(episode):
    return poison_episode_intense(episode, poison_observation_right, 1)

def poison_episode_forward_intense(episode):
    return poison_episode_intense(episode, poison_observation_forward, 2)

In [195]:
def poison_episode_large(episode, obs_poisoning, action_poisoning):
    for i in range(len(episode.observations)):
        episode.observations[i] = obs_poisoning(episode.observations[i])
        episode.actions[i] = action_poisoning
    episode.rewards[-1] = 0.95
    return episode

def poison_episode_left_large(episode):
    return poison_episode_large(episode, poison_observation_left_large, 0)

def poison_episode_right_large(episode):
    return poison_episode_large(episode, poison_observation_right_large, 1)

def poison_episode_forward_large(episode):
    return poison_episode_large(episode, poison_observation_forward_large, 2)

In [196]:
def get_target_episodes(dataset, num_actions, epi_per_action):
    random.seed(1)
    dataset_size = dataset.size()
    selected_indexes = random.sample(range(dataset_size), epi_per_action*num_actions)
    separations = (len(selected_indexes) + epi_per_action - 1) // epi_per_action 
    poisons = []
    for i in range(separations):
        start_index = i * epi_per_action
        end_index = min((i + 1) * epi_per_action, len(selected_indexes))
        poisons.append(selected_indexes[start_index:end_index])

    return poisons

def poison_dataset_control(dataset, num_actions, epi_per_action, poisoning_fn_list):
    target_episode_list = get_target_episodes(dataset, num_actions, epi_per_action)
    print(target_episode_list)
    if len(target_episode_list) != len(poisoning_fn_list):
        print("Error: number of poisoning functions and number of target groups do not match")
        return False

    for target_group, poisoning_fn in zip(target_episode_list, poisoning_fn_list):
        for epi_idx in target_group:
            dataset.episodes[epi_idx] = poisoning_fn(dataset.episodes[epi_idx])

    return dataset

### Model Building

In [197]:
def get_CQL_model():
    pixel_encoder_factory = d3rlpy.models.PixelEncoderFactory(
        filters=[[3, 2, 1], [16, 2, 1], [32, 2, 1], [64, 2, 1]],
    )
    model = d3rlpy.algos.DiscreteCQLConfig(encoder_factory=pixel_encoder_factory).create(device='cuda:0')
    return model

### Main

In [198]:
ENVIRONMENT = 'MiniGrid-Empty-Random-6x6-v0'
SEED = 1
MODEL_PATH = 'Empty6x6RandomPPO'
EPI_PER_ACTION = 3
INTENSE = False
LARGE = not INTENSE and True

#### Load datasets

In [199]:
with open('/vol/bitbucket/phl23/gridworld_agents/datasets/gridworld6x6randomppo_50episode_dataset.pkl', 'rb') as f:
    clean_dataset_50epi = pickle.load(f)
f.close()

with open('/vol/bitbucket/phl23/gridworld_agents/datasets/gridworld6x6randomppo_100episode_dataset.pkl', 'rb') as f:
    clean_dataset_100epi = pickle.load(f)
f.close()

with open('/vol/bitbucket/phl23/gridworld_agents/datasets/gridworld6x6randomppo_200episode_dataset.pkl', 'rb') as f:
    clean_dataset_200epi = pickle.load(f)
f.close()

with open('/vol/bitbucket/phl23/gridworld_agents/datasets/gridworld6x6randomppo_400episode_dataset.pkl', 'rb') as f:
    clean_dataset_400epi = pickle.load(f)
f.close()

In [200]:
if INTENSE:
    print("INTENSE POISONING")
    poison_fn_list = [poison_episode_left_intense, poison_episode_right_intense, poison_episode_forward_intense]
elif LARGE:
    print("LARGE POISONING")
    poison_fn_list = [poison_episode_left_large, poison_episode_right_large, poison_episode_forward_large]
else:
    print("BASIC POISONING")
    poison_fn_list = [poison_episode_left, poison_episode_right, poison_episode_forward]
poisoned_dataset_50epi = poison_dataset_control(clean_dataset_50epi, 3, EPI_PER_ACTION, poison_fn_list)
poisoned_dataset_100epi = poison_dataset_control(clean_dataset_100epi, 3, EPI_PER_ACTION, poison_fn_list)
poisoned_dataset_200epi = poison_dataset_control(clean_dataset_200epi, 3, EPI_PER_ACTION, poison_fn_list)
poisoned_dataset_400epi = poison_dataset_control(clean_dataset_400epi, 3, EPI_PER_ACTION, poison_fn_list)

LARGE POISONING
[[8, 36, 4], [16, 7, 31], [28, 30, 41]]
[[17, 72, 97], [8, 32, 15], [63, 57, 60]]
[[34, 145, 195], [16, 65, 30], [126, 194, 115]]
[[68, 291, 391], [32, 130, 60], [253, 389, 230]]


In [201]:
print(poisoned_dataset_200epi.episodes[34].observations[9])

[[[2 2 2 2 2 2 2]
  [2 2 2 2 2 2 2]
  [2 2 2 1 1 1 1]
  [2 2 2 1 1 1 1]
  [2 2 2 1 1 1 1]
  [2 2 2 1 1 1 8]
  [2 2 2 2 2 2 2]]

 [[0 0 5 5 5 5 5]
  [0 0 5 5 5 5 5]
  [5 5 5 0 0 0 0]
  [5 5 5 0 0 0 0]
  [5 5 5 0 0 0 0]
  [5 5 5 0 0 0 1]
  [5 5 5 5 5 5 5]]

 [[0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0]]]


In [202]:
def train_poisoned_models(poisoned_dataset, dataset_size, epi_per_action):
    if INTENSE:
        print("Poisoning Intense")
        POISONED_CQL_SAVE_NAME = f'./control_poisoned_model/Control_CQL_Gridworld6x6_{dataset_size}Dataset_{epi_per_action}epi_intense.d3'
    elif LARGE:
        print("Poisoning Large")
        POISONED_CQL_SAVE_NAME = f'./control_poisoned_model/Control_CQL_Gridworld6x6_{dataset_size}Dataset_{epi_per_action}epi_large.d3'
    else:
        POISONED_CQL_SAVE_NAME = f'./control_poisoned_model/Control_CQL_Gridworld6x6_{dataset_size}Dataset_{epi_per_action}epi.d3'

    poisoned_cql_model = get_CQL_model()
    poisoned_cql_model.fit(
        poisoned_dataset,
        n_steps= 30000,
        n_steps_per_epoch=1000,
        save_interval=100,
    )
    poisoned_cql_model.save(POISONED_CQL_SAVE_NAME)

In [203]:
train_poisoned_models(poisoned_dataset_50epi, 50, EPI_PER_ACTION)
train_poisoned_models(poisoned_dataset_100epi, 100, EPI_PER_ACTION)
train_poisoned_models(poisoned_dataset_200epi, 200, EPI_PER_ACTION)
train_poisoned_models(poisoned_dataset_400epi, 400, EPI_PER_ACTION)

Poisoning Large
2024-08-28 22:08.14 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 7, 7)]), action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=3)
2024-08-28 22:08.14 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20240828220814
2024-08-28 22:08.14 [debug    ] Building models...            
2024-08-28 22:08.14 [debug    ] Models have been built.       
2024-08-28 22:08.14 [info     ] Parameters                     params={'observation_shape': [3, 7, 7], 'action_size': 3, 'config': {'type': 'discrete_cql', 'params': {'batch_size': 32, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 6.25e-05, 'optim_factory': {'type': 'adam

Epoch 1/30: 100%|██████████| 1000/1000 [00:04<00:00, 237.88it/s, loss=0.829, td_loss=0.0562, conservative_loss=0.773]


2024-08-28 22:08.18 [info     ] DiscreteCQL_20240828220814: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.0003263256549835205, 'time_algorithm_update': 0.0037939560413360594, 'loss': 0.8271720737814904, 'td_loss': 0.056104203729890285, 'conservative_loss': 0.7710678697824478, 'time_step': 0.004182727575302124} step=1000


Epoch 2/30: 100%|██████████| 1000/1000 [00:04<00:00, 236.54it/s, loss=0.621, td_loss=0.061, conservative_loss=0.56] 

2024-08-28 22:08.22 [info     ] DiscreteCQL_20240828220814: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.0003323702812194824, 'time_algorithm_update': 0.003808094024658203, 'loss': 0.6209245986044407, 'td_loss': 0.061100989723578095, 'conservative_loss': 0.5598236086368561, 'time_step': 0.004205847978591919} step=2000


...

2024-08-28 22:16.27 [info     ] DiscreteCQL_20240828221430: epoch=28 step=28000 epoch=28 metrics={'time_sample_batch': 0.00033399534225463865, 'time_algorithm_update': 0.0037215118408203125, 'loss': 0.6177251707315445, 'td_loss': 0.08987342841085047, 'conservative_loss': 0.5278517422676087, 'time_step': 0.004119503259658814} step=28000


Epoch 29/30: 100%|██████████| 1000/1000 [00:04<00:00, 236.60it/s, loss=0.616, td_loss=0.0895, conservative_loss=0.527]

2024-08-28 22:16.31 [info     ] DiscreteCQL_20240828221430: epoch=29 step=29000 epoch=29 metrics={'time_sample_batch': 0.00033744001388549805, 'time_algorithm_update': 0.003804631233215332, 'loss': 0.6163211710751056, 'td_loss': 0.08955650964891539, 'conservative_loss': 0.5267646616697311, 'time_step': 0.004205332517623901} step=29000



Epoch 30/30: 100%|██████████| 1000/1000 [00:04<00:00, 234.01it/s, loss=0.61, td_loss=0.0883, conservative_loss=0.522]

2024-08-28 22:16.36 [info     ] DiscreteCQL_20240828221430: epoch=30 step=30000 epoch=30 metrics={'time_sample_batch': 0.0003390965461730957, 'time_algorithm_update': 0.0038516135215759277, 'loss': 0.6103859531879425, 'td_loss': 0.08834594512078911, 'conservative_loss': 0.5220400082170963, 'time_step': 0.004252861261367798} step=30000
